# Standard map plots

In [15]:
import glob
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LogNorm

In [17]:
OUTDIR = "../src/classical/out_standardmap"
FIGDIR = os.path.join("../figs/classical")
os.makedirs(FIGDIR, exist_ok=True)
plt.style.use("science.mplstyle")

## ✅ Different K values multiplots ✅

In [18]:
def k_to_str(K):
    return f"{K:.3f}"


def read_poincare_orbits(path):
    """Read blocks separated by blank lines: each block is one orbit."""
    orbits = []
    cur = []
    with open(path, "r") as f:
        for line in f:
            s = line.strip()
            if (not s) or s.startswith("#"):
                if cur:
                    orbits.append(np.array(cur, dtype=float))
                    cur = []
                continue
            th, p = map(float, s.split())
            cur.append((th, p))
    if cur:
        orbits.append(np.array(cur, dtype=float))
    return orbits


def load_ftle_grid(K):
    path = os.path.join(OUTDIR, f"ftle_grid_K_{k_to_str(K)}.csv")
    df = pd.read_csv(path)
    thetas = np.sort(df["theta0"].unique())
    ps = np.sort(df["p0"].unique())
    Z = df.pivot(index="p0", columns="theta0", values="ftle").to_numpy()
    return thetas, ps, Z


def compute_global_ftle_scale(K_list, percentile=99.0):
    vals = []
    for K in K_list:
        _, _, Z = load_ftle_grid(K)
        z = Z[np.isfinite(Z)]
        vals.append(z)
    allv = np.concatenate(vals) if vals else np.array([0.0])
    vmin = 0.0
    vmax = float(np.nanpercentile(allv, percentile))
    if vmax <= vmin:
        vmax = vmin + 1e-6
    return vmin, vmax


def draw_ftle_imshow(ax, thetas, ps, Z, cmap, norm):
    """
    FTLE map with pixel-crisp rendering (no interpolation). 
    """
    extent = (thetas.min(), thetas.max(), ps.min(), ps.max())
    im = ax.imshow(
        Z,
        origin="lower",
        extent=extent,
        cmap=cmap,
        norm=norm,
        interpolation="none",   # sharp pixels 
        aspect="auto",
        zorder=1
    )
    return im


def overlay_poincare_sharp_scatter(ax, K, thetas, ps, Z,
                                  ftle_cut=1e-3,
                                  max_points=250000,
                                  chaotic_step=80,
                                  regular_s=1.0,
                                  regular_alpha=0.85,
                                  chaotic_s=6.0,
                                  chaotic_alpha=0.06):
    """
    Paper-like overlay using only points (no lines), but sharp:
    - square markers, no edge/linewidth artifacts
    - rasterized points for crisp output and manageable files 
    """
    poinc_path = os.path.join(OUTDIR, f"poincare_K_{k_to_str(K)}.dat")
    if not os.path.exists(poinc_path):
        return

    orbits = read_poincare_orbits(poinc_path)
    if not orbits:
        return

    pts = np.concatenate(orbits, axis=0)
    th = pts[:, 0]
    p = pts[:, 1]

    # subsample for speed / density control
    if th.size > max_points:
        idx = np.random.choice(th.size, size=max_points, replace=False)
        th = th[idx]
        p = p[idx]

    Nt = len(thetas)
    Np = len(ps)
    it = np.clip((th / (2*np.pi) * Nt).astype(int), 0, Nt - 1)
    ip = np.clip((p  / (2*np.pi) * Np).astype(int), 0, Np - 1)
    ft = Z[ip, it]

    ok = np.isfinite(ft)
    th = th[ok]
    p = p[ok]
    ft = ft[ok]

    regular = ft < ftle_cut
    chaotic = ~regular

    # Regular: dense and sharp
    ax.scatter(
        th[regular], p[regular],
        s=regular_s, marker="s",
        c="white", alpha=regular_alpha,
        linewidths=0, edgecolors="none",
        rasterized=True,
        zorder=3
    )

    # Chaotic: sparse
    idx = np.where(chaotic)[0]
    if idx.size:
        idx2 = idx[::chaotic_step]
        ax.scatter(
            th[idx2], p[idx2],
            s=chaotic_s, marker="s",
            c="white", alpha=chaotic_alpha,
            linewidths=0, edgecolors="none",
            rasterized=True,
            zorder=2
        )


In [19]:
# K snapshots: ORIGINAL PAPER'S 6 PLOTS 
K_PAPER = [0.2, 0.5, 0.8, 0.971, 1.8, 5.5]

def paper_multiplot_poincare_jpg_A_hist2d(
    bins=500,
    log_vmin=1,
):
    """
    Poincaré as density image via 2D histogram + LogNorm.
    Black = high density; white = low density.
    """
    fig = plt.figure(figsize=(11.0, 6.6), dpi=600)
    gs = GridSpec(2, 3, figure=fig, wspace=0.10, hspace=0.10)

    for i, K in enumerate(K_PAPER):
        r, c = divmod(i, 3)
        ax = fig.add_subplot(gs[r, c])

        f = os.path.join(OUTDIR, f"poincare_K_{k_to_str(K)}.dat")
        if os.path.exists(f):
            data = np.loadtxt(f, comments="#")
            if data.ndim == 2 and data.shape[0] > 0:
                theta, p = data[:, 0], data[:, 1]

                H, xedges, yedges = np.histogram2d(
                    theta, p,
                    bins=bins,
                    range=[[0, 2*np.pi], [0, 2*np.pi]]
                )
                H = H.T  # for imshow (y,x)

                # Display density (white background, black density)
                cmapP = plt.get_cmap("gray_r").copy()
                cmapP.set_over("black")  

                vmaxP = np.percentile(H[H > 0], 99.7)  
                ax.imshow(
                    H,
                    origin="lower",
                    extent=(0, 2*np.pi, 0, 2*np.pi),
                    cmap=cmapP,
                    norm=LogNorm(vmin=log_vmin, vmax=max(log_vmin, vmaxP), clip=False),
                    interpolation="nearest",
                    aspect="auto",
                    zorder=1
                )


        ax.set_xlim(0, 2*np.pi)
        ax.set_ylim(0, 2*np.pi)

        if c == 0:
            ax.set_ylabel(r"$p$")
        else:
            ax.set_yticklabels([])

        if r == 1:
            ax.set_xlabel(r"$\theta$")
        else:
            ax.set_xticklabels([])

    fig.savefig(os.path.join(FIGDIR, "paper_multiplot_poincare.pdf"), bbox_inches="tight")
    plt.close(fig)

def paper_multiplot_ftle_overlay_jpg_A_hist2d(
    K_list,
    colormap="inferno",
    ftle_percentile=99.0,
    bins_overlay=450,
    overlay_alpha=0.7,   
    log_vmin=1,
    count_clip=3,       
):
    """
    FTLE background (pcolormesh) + Poincaré density overlay (hist2d + LogNorm).
      - Cells with low density become fully transparent.
      - The densest ones reach pure white.
    """
    vmin, vmax = compute_global_ftle_scale(K_list, percentile=ftle_percentile)
    norm = mcolors.Normalize(vmin=vmin, vmax=vmax)

    fig = plt.figure(figsize=(11.6, 6.6), dpi=600)
    gs = GridSpec(2, 4, figure=fig, width_ratios=[1, 1, 1, 0.06],
                  wspace=0.10, hspace=0.10)

    mappable0 = None

    for i, K in enumerate(K_PAPER):
        r, c = divmod(i, 3)
        ax = fig.add_subplot(gs[r, c])

        # --- FTLE map background ---
        f = os.path.join(OUTDIR, f"ftle_grid_K_{k_to_str(K)}.csv")
        if os.path.exists(f):
            df = pd.read_csv(f)
            thetas = np.sort(df["theta0"].unique())
            ps     = np.sort(df["p0"].unique())
            Z = df.pivot(index="p0", columns="theta0", values="ftle").to_numpy()

            mesh = ax.pcolormesh(thetas, ps, Z, shading="auto",
                                 cmap=colormap, norm=norm, antialiased=False)
            if mappable0 is None:
                mappable0 = mesh

        # --- Overlay Poincaré density ---
        poinc_path = os.path.join(OUTDIR, f"poincare_K_{k_to_str(K)}.dat")
        if os.path.exists(poinc_path):
            data_p = np.loadtxt(poinc_path, comments="#")
            if data_p.ndim == 2 and data_p.shape[0] > 0:
                thp, pp = data_p[:, 0], data_p[:, 1]

                H, _, _ = np.histogram2d(
                    thp, pp,
                    bins=bins_overlay,
                    range=[[0, 2*np.pi], [0, 2*np.pi]]
                )
                H = H.T

                # 1) cuts sound: cells with few points 
                if count_clip is not None and count_clip > 0:
                    Hmask = np.ma.masked_less_equal(H, count_clip)
                else:
                    Hmask = np.ma.masked_equal(H, 0)

                # 2) colormap: pure white at high density, transparency in masked
                cmapO = plt.get_cmap("gray").copy()
                cmapO.set_over("white")          # pure white
                cmapO.set_bad((0, 0, 0, 0))      # masked (noise)

                if np.any(Hmask > 0):
                    vmaxO = np.percentile(Hmask.compressed(), 99.7)
                else:
                    vmaxO = 1.0

                ax.imshow(
                    Hmask,
                    origin="lower",
                    extent=(0, 2*np.pi, 0, 2*np.pi),
                    cmap=cmapO,
                    norm=LogNorm(vmin=log_vmin, vmax=max(log_vmin, vmaxO), clip=False),
                    interpolation="nearest",
                    alpha=overlay_alpha,  # alpha for the FTLE overlay
                    aspect="auto",
                    zorder=3
                )

        ax.set_xlim(0, 2*np.pi)
        ax.set_ylim(0, 2*np.pi)

        if c == 0:
            ax.set_ylabel(r"$p_0$")
        else:
            ax.set_yticklabels([])

        if r == 1:
            ax.set_xlabel(r"$\theta_0$")
        else:
            ax.set_xticklabels([])

    cax = fig.add_subplot(gs[:, 3])
    if mappable0 is not None:
        cbar = fig.colorbar(mappable0, cax=cax)
        cbar.set_label("FTLE / Lyapunov (finite-time)")

    fig.savefig(os.path.join(FIGDIR, "paper_multiplot_ftle_overlay.pdf"),
                bbox_inches="tight")
    plt.close(fig)


In [20]:
# paper version
if __name__ == "__main__":
    paper_multiplot_poincare_jpg_A_hist2d()
    paper_multiplot_ftle_overlay_jpg_A_hist2d(K_PAPER)
    print(f"Saved multiplots into: {FIGDIR}")

Saved multiplots into: ../figs/classical


In [21]:
# MORE K snapshots: 8 plots
K_PAPER = [0.2, 0.5, 0.8, 0.971, 1.4, 1.8, 3.0, 5.5]

def paper_multiplot_poincare_jpg_A_hist2d(
    bins=500,
    log_vmin=1,
):
    """
    Poincaré as density image via 2D histogram + LogNorm.
    Black = high density; white = low density.
    """
    fig = plt.figure(figsize=(12.8, 6.6), dpi=600)
    gs = GridSpec(2, 4, figure=fig, wspace=0.10, hspace=0.10)

    for i, K in enumerate(K_PAPER):
        r, c = divmod(i, 4)
        ax = fig.add_subplot(gs[r, c])

        f = os.path.join(OUTDIR, f"poincare_K_{k_to_str(K)}.dat")
        if os.path.exists(f):
            data = np.loadtxt(f, comments="#")
            if data.ndim == 2 and data.shape[0] > 0:
                theta, p = data[:, 0], data[:, 1]

                H, _, _ = np.histogram2d(
                    theta, p,
                    bins=bins,
                    range=[[0, 2*np.pi], [0, 2*np.pi]]
                )
                H = H.T  # for imshow (y,x)

                cmapP = plt.get_cmap("gray_r").copy()
                cmapP.set_over("black")

                if np.any(H > 0):
                    vmaxP = np.percentile(H[H > 0], 99.7)
                else:
                    vmaxP = log_vmin

                ax.imshow(
                    H,
                    origin="lower",
                    extent=(0, 2*np.pi, 0, 2*np.pi),
                    cmap=cmapP,
                    norm=LogNorm(vmin=log_vmin, vmax=max(log_vmin, vmaxP), clip=False),
                    interpolation="nearest",
                    aspect="auto",
                    zorder=1
                )

        ax.set_xlim(0, 2*np.pi)
        ax.set_ylim(0, 2*np.pi)

        # Labels
        if c == 0:
            ax.set_ylabel(r"$p$")
        else:
            ax.set_yticklabels([])

        if r == 1:
            ax.set_xlabel(r"$\theta$")
        else:
            ax.set_xticklabels([])

        # Optional: tiny title per panel
        ax.set_title(f"K = {K:g}", fontsize=9)

    fig.savefig(os.path.join(FIGDIR, "8_multiplot_poincare.pdf"), bbox_inches="tight")
    plt.close(fig)


def paper_multiplot_ftle_overlay_jpg_A_hist2d(
    K_list,
    colormap="inferno",
    ftle_percentile=99.0,
    bins_overlay=450,
    overlay_alpha=0.7,
    log_vmin=1,
    count_clip=3,
):
    """
    FTLE background (pcolormesh) + Poincaré density overlay (hist2d + LogNorm).
    - Cel·les amb densitat baixa es fan totalment transparents (masked).
    - Les més denses arriben a blanc pur.
    """
    vmin, vmax = compute_global_ftle_scale(K_list,percentile=ftle_percentile)
    norm = mcolors.Normalize(vmin=vmin, vmax=vmax)

    fig = plt.figure(figsize=(13.4, 6.6), dpi=600)
    gs = GridSpec(
        2, 5, figure=fig,
        width_ratios=[1, 1, 1, 1, 0.06],
        wspace=0.10, hspace=0.10
    )

    mappable0 = None

    for i, K in enumerate(K_PAPER):
        r, c = divmod(i, 4)
        ax = fig.add_subplot(gs[r, c])

        # --- FTLE map de fons ---
        f = os.path.join(OUTDIR, f"ftle_grid_K_{k_to_str(K)}.csv")
        if os.path.exists(f):
            df = pd.read_csv(f)
            thetas = np.sort(df["theta0"].unique())
            ps     = np.sort(df["p0"].unique())
            Z = df.pivot(index="p0", columns="theta0", values="ftle").to_numpy()

            mesh = ax.pcolormesh(
                thetas, ps, Z,
                shading="auto",
                cmap=colormap,
                norm=norm,
                antialiased=False
            )
            if mappable0 is None:
                mappable0 = mesh

        # --- Overlay de densitat Poincaré ---
        poinc_path = os.path.join(OUTDIR, f"poincare_K_{k_to_str(K)}.dat")
        if os.path.exists(poinc_path):
            data_p = np.loadtxt(poinc_path, comments="#")
            if data_p.ndim == 2 and data_p.shape[0] > 0:
                thp, pp = data_p[:, 0], data_p[:, 1]

                H, _, _ = np.histogram2d(
                    thp, pp,
                    bins=bins_overlay,
                    range=[[0, 2*np.pi], [0, 2*np.pi]]
                )
                H = H.T

                if count_clip is not None and count_clip > 0:
                    Hmask = np.ma.masked_less_equal(H, count_clip)
                else:
                    Hmask = np.ma.masked_equal(H, 0)

                cmapO = plt.get_cmap("gray").copy()
                cmapO.set_over("white")
                cmapO.set_bad((0, 0, 0, 0))  # transparent for masked

                if np.any(Hmask > 0):
                    vmaxO = np.percentile(Hmask.compressed(), 99.7)
                else:
                    vmaxO = 1.0

                ax.imshow(
                    Hmask,
                    origin="lower",
                    extent=(0, 2*np.pi, 0, 2*np.pi),
                    cmap=cmapO,
                    norm=LogNorm(vmin=log_vmin, vmax=max(log_vmin, vmaxO), clip=False),
                    interpolation="nearest",
                    alpha=overlay_alpha,
                    aspect="auto",
                    zorder=3
                )

        ax.set_xlim(0, 2*np.pi)
        ax.set_ylim(0, 2*np.pi)

        # Labels
        if c == 0:
            ax.set_ylabel(r"$p_0$")
        else:
            ax.set_yticklabels([])

        if r == 1:
            ax.set_xlabel(r"$\theta_0$")
        else:
            ax.set_xticklabels([])

        ax.set_title(f"K = {K:g}", fontsize=9)

    # --- Colorbar column ---
    cax = fig.add_subplot(gs[:, 4])
    if mappable0 is not None:
        cbar = fig.colorbar(mappable0, cax=cax)
        cbar.set_label("FTLE / Lyapunov (finite-time)")

    fig.savefig(os.path.join(FIGDIR, "8_multiplot_ftle_overlay.pdf"), bbox_inches="tight")
    plt.close(fig)


In [22]:
# poster version
if __name__ == "__main__":
    paper_multiplot_poincare_jpg_A_hist2d()
    paper_multiplot_ftle_overlay_jpg_A_hist2d(K_PAPER)
    print(f"Saved multiplots into: {FIGDIR}")

Saved multiplots into: ../figs/classical


## ✅ Lyapunov coefficient: interesting K region ✅

In [23]:
# --- critical value (golden KAM curve breakup) ---
Kc = 0.971635  # (aprox)

df = pd.read_csv(os.path.join(OUTDIR, "lyapunov_vs_K.csv"))
for col in ["K", "lambda_mean", "lambda_std"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")
df = df.dropna(subset=["K", "lambda_mean"]).sort_values("K")

K = df["K"].to_numpy()
lam = df["lambda_mean"].to_numpy()
err = df["lambda_std"].fillna(0.0).to_numpy()

# point of our simulation closest to Kc
iKc = int(np.argmin(np.abs(K - Kc)))
Kc_data = K[iKc]
lam_Kc = lam[iKc]

fig, ax = plt.subplots(figsize=(4.2, 3.4), dpi=600)

main_color = "#3b528b"
kc_color = "#5ec962"
ms_all = 2  # point size

# Shadow +- 1 sigma, with fillbetween
ax.fill_between(
    K, lam - err, lam + err,
    color=main_color, alpha=0.18, linewidth=0, zorder=1
)   

# Line+points (all same size)
ax.plot(K, lam, "o--", ms=ms_all, lw=1, color=main_color, zorder=3)

ax.set_xlabel("K")
ax.set_ylabel(r"$\lambda$")
ax.set_ylim(bottom=0)
ax.grid(True, alpha=0.25)

# --- Highlight Kc ---
ax.plot([Kc_data], [lam_Kc], "o", ms=ms_all+2, color=kc_color, alpha=0.35, zorder=4)
ax.plot([Kc_data], [lam_Kc], "o", ms=ms_all, color=kc_color, zorder=5)

# --- ZOOM ---
x1, x2 = 0.85, 1.05
mask = (K >= x1) & (K <= x2)
y1 = np.nanmin((lam - err)[mask]) if np.any(mask) else np.nanmin(lam - err)
y2 = np.nanmax((lam + err)[mask]) if np.any(mask) else np.nanmax(lam + err)
pad = 0.08 * (y2 - y1 if y2 > y1 else 1.0)
y1, y2 = max(0.0, y1 - pad), y2 + pad

axins = ax.inset_axes([0.10, 0.50, 0.38, 0.40])

axins.fill_between(
    K, lam - err, lam + err,
    color=main_color, alpha=0.18, linewidth=0, zorder=1
)

axins.plot(K, lam, "o--", ms=ms_all+1, lw=1.4, color=main_color, zorder=3)

axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.grid(True, alpha=0.20)
axins.tick_params(labelleft=False, labelbottom=False)

axins.set_title(
    rf"$K_c \approx {Kc:.6f}$",
    fontsize=8.5
)

# Show Kc in zoom
axins.plot([Kc_data], [lam_Kc], "o", ms=ms_all+3, color=kc_color, alpha=0.35, zorder=4)
axins.plot([Kc_data], [lam_Kc], "o", ms=ms_all+1, color=kc_color, zorder=5)

# Square + connectors
try:
    ax.indicate_inset_zoom(axins, edgecolor="0.25", lw=1.2)
except AttributeError:
    from mpl_toolkits.axes_grid1.inset_locator import mark_inset
    mark_inset(ax, axins, loc1=2, loc2=4, fc="none", ec="0.25", lw=1.2)

fig.tight_layout()
fig.savefig(os.path.join(FIGDIR, "lyapunov_vs_K_band_inset.pdf"), bbox_inches="tight")
plt.close(fig)


## Classic difussion

In [24]:
# --- MSD / diffusion (cylinder, unwrapped p) helpers ---

def load_msd(K):
    path = os.path.join(OUTDIR, f"msd_K_{k_to_str(K)}.csv")
    df = pd.read_csv(path)
    for c in ["n", "msd"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df = df.dropna(subset=["n", "msd"]).sort_values("n")
    return df


def fit_linear_slope(n, y, nmin=500, nmax=None):
    # Fit y ≈ a + b n on a window (ignore early transient)
    n = np.asarray(n, dtype=float)
    y = np.asarray(y, dtype=float)
    if nmax is None:
        nmax = float(np.max(n))
    mask = (n >= nmin) & (n <= nmax) & np.isfinite(y)
    x = n[mask]
    yy = y[mask]
    if x.size < 10:
        raise ValueError("Not enough points for linear fit.")
    b, a = np.polyfit(x, yy, 1)  # y = a + b n
    yhat = a + b * x
    ssres = float(np.sum((yy - yhat) ** 2))
    sstot = float(np.sum((yy - np.mean(yy)) ** 2))
    r2 = 1.0 - ssres / sstot if sstot > 0 else np.nan
    return float(a), float(b), float(r2), int(x.size)


In [25]:
# --- Single-K diffusion plot (poster bridge) ---

Kdiff = 5.5  # an example value: big K
df = load_msd(Kdiff)

n = df["n"].to_numpy(dtype=int)
msd = df["msd"].to_numpy(dtype=float)

# Fit slope in a late-time window
a, b, r2, npts = fit_linear_slope(n, msd, nmin=800, nmax=5000)

fig, ax = plt.subplots(figsize=(4.2, 3.4), dpi=900)

main_color = "#3b528b"
fit_color = "#5ec962"

ax.plot(n, msd, lw=1.6, color=main_color, zorder=3)

# Fit line overlay
xfit = np.array([n.min(), n.max()], dtype=float)
yfit = a + b * xfit
ax.plot(xfit, yfit, ls="--", lw=1.4, color=fit_color, zorder=4)

ax.set_xlabel("Number of kicks n")
ax.set_ylabel(r"$\langle (p-p_0)^2 \rangle$")
ax.grid(True, alpha=0.25)

# Small text box 
ax.text(
    0.02, 0.98,
    rf"$K={Kdiff}$" + "\n" + rf"Slope $\approx {b:.3e}$" + "\n" + rf"$R^2={r2:.3f}$",
    transform=ax.transAxes,
    va="top", ha="left",
    fontsize=8,
    color="0.25",
)

fig.tight_layout()
fig.savefig(os.path.join(FIGDIR, f"msd_vs_n_K_{k_to_str(Kdiff)}.pdf"), bbox_inches="tight")
plt.close(fig)
